# 1. 
    1.1 模型处理
        1.2 获取数据
        1.3 模型
        1.4 多个模型
        1.5 融合
        

In [1]:
import jieba 
import gensim
import pandas as pd
import numpy as np
import jieba.analyse
import lightgbm
import sklearn
import torch
from torch import nn
from sklearn.metrics import f1_score
import torch.nn.functional as F

F:\app\anaconda\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
from batch_size import datasets
from torch.utils.data import Dataset, DataLoader
from preprocess import pre_train
import config as c
word2vec_model=gensim.models.KeyedVectors.load_word2vec_format(c.word2vec_file)
load_word=pre_train()
data=load_word.load_word()

In [116]:
def get_data(idx):
    '''
    就是把数据传入啊
    '''
    q1=np.zeros((len(idx),128))
    q2=np.zeros((len(idx),128))    
    for i,j in enumerate(idx):        
        q1[i]=transform_word_vector(data['q1'][j-1])
        q2[i]=transform_word_vector(data['q2'][j-1])        
    target=np.array([data['target'][i-1] for i in idx])
    return q1,q2,target 

def transform_word_vector(word_lists):
    '''
    分好词的句子的列表
    '''
    re=np.zeros((len(word_lists),128))
    for i,word in enumerate(word_lists):
        try:
            re[i]=word2vec_model[word]
        except:
            pass 
    return re.mean(axis=0)

In [122]:
dataset=datasets()
datas=DataLoader(dataset,batch_size=40000)

### 2. lightgbm

In [7]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 5,
    'learning_rate': 0.10,
    'feature_fraction': 0.7,
    'bagging_fraction': 0.8,
    'bagging_freq': 3,    
    'reg_lambda':0.9,    
    'subsample':0.8,
    'subsample_freq':30,  
}

In [124]:
for i,idx in enumerate(datas):
    q1,q2,train_y=get_data(idx.numpy())
    train_x=np.hstack([q1,q2])

    lgb_train = lightgbm.Dataset(train_x, train_y,free_raw_data=True)    
    if i==0:
        model_name=None
    else:
        model_name=estimator
    estimator = lightgbm.train(params,
                    lgb_train,
                    num_boost_round=300,
                    valid_sets=lgb_train,
                    init_model=model_name,
                    keep_training_booster=False,
                    early_stopping_rounds=50,
                    verbose_eval=150,
                    )
    y=estimator.predict(train_x).reshape(-1,1)
    y_pred=[int(i>0.16) for i in y]
    print('f1_score',i,f1_score(train_y,y_pred))


Training until validation scores don't improve for 50 rounds.
[150]	training's auc: 0.717622
[300]	training's auc: 0.775539
Did not meet early stopping. Best iteration is:
[300]	training's auc: 0.775539
f1_scire 0 0.4149298597194389
Training until validation scores don't improve for 50 rounds.
[450]	training's auc: 0.806446
[600]	training's auc: 0.836729
Did not meet early stopping. Best iteration is:
[600]	training's auc: 0.836729
f1_scire 1 0.4557804269476019
Training until validation scores don't improve for 50 rounds.
[750]	training's auc: 0.775617
[900]	training's auc: 0.838107
Did not meet early stopping. Best iteration is:
[900]	training's auc: 0.838107
f1_scire 2 0.47715247715247716


* 保存模型

In [125]:
estimator.save_model('../model/lightgbm_model')

* load 模型

In [ ]:
bst = lightgbm.Booster(model_file='../model/lightgbm_model')

* 训练的结果导致需要一个阈值来保证最大化
        如何求阈值？

* 将两列合并之后再训练
   
       1.1 lightgbm 简单
       1.2 线性呢？
       1.3 torch神经网络？

* 先训练，然后在合并